In [1]:
import stanza
import pandas as pd
import numpy as np

import re
import spacy

# Reading Data

In [2]:
df_mex_her = pd.read_excel("../data/Mexico/Heraldo_scraped.xlsx")
df_mex_her = df_mex_her.sample(n=300, random_state=1996)

df_mex_elu = pd.read_excel("../data/Mexico/el_universal_scraped.xlsx")
df_mex_elu = df_mex_elu.sample(n=300, random_state=1996)

df_mex_laj = pd.read_excel("../data/Mexico/la_jornada_scraped.xlsx")
df_mex_laj = df_mex_laj.sample(n=300, random_state=1996)

df_mex_comb = df_mex_her.append(df_mex_elu).append(df_mex_laj)

df_uk_sun = pd.read_csv("../data/UK/sun_scraped_cleaned.csv")
df_uk_sun = df_uk_sun.sample(n=300, random_state=1996)

df_uk_tim = pd.read_csv("../data/UK/times_scraped_cleaned.csv")
df_uk_tim = df_uk_tim.sample(n=300, random_state=1996)

df_uk_gua = pd.read_csv("../data/UK/guardian_scraped_cleaned.csv")
df_uk_gua = df_uk_gua.sample(n=300, random_state=1996)

df_uk_gua['title'] = df_uk_gua['webTitle']
df_uk_gua['text'] = df_uk_gua['body']

df_uk_comb = df_uk_sun.append(df_uk_tim).append(df_uk_gua)

df_pak_comb = pd.read_csv("../data/Pakistan/pak_extracted_combined_2020.csv")






/Users/rukhshan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


# English dependency parser
## Function

In [3]:
dep_parser_en = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

def check_if_passive_en(sentence, dep_parser, method1=True):
    '''
    ADD DOCSTRING
    '''


    if sentence:
        sentence = str(sentence)
        try:
            doc = dep_parser(sentence)
            num_passive = 0
            sent_lst = []

            for sent in doc.sentences:
                for word in sent.words:
                    deprel = word.deprel
                    if "pass" in deprel:
                        num_passive += 1
                        sent_lst.append(sent)

            return [num_passive, sent_lst]

        except:
            pass
    else:
        return [None, None]
    



2021-05-24 00:57:45 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-05-24 00:57:45 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2021-05-24 00:57:45 INFO: Use device: cpu
2021-05-24 00:57:45 INFO: Loading: tokenize
2021-05-24 00:57:45 INFO: Loading: pos
2021-05-24 00:57:45 INFO: Loading: lemma
2021-05-24 00:57:45 INFO: Loading: depparse
2021-05-24 00:57:46 INFO: Done loading processors!


## Method 1

In [4]:
df_pak_comb['combined_text'] = df_pak_comb['title'] + df_pak_comb['text']
df_uk_comb['combined_text'] = df_uk_comb['title'] + df_uk_comb['text']

In [29]:
print("Getting num_passive for Pakistan")
df_pak_comb = df_pak_comb
df_pak_comb['passive_instances'] = df_pak_comb['combined_text'].apply(check_if_passive_en, dep_parser=dep_parser_en)
df_pak_comb.to_excel("../data/Pakistan/pakistan_stanza_passive.xlsx")




Getting num_passive for Pakistan
Getting num_passive for the UK


RuntimeError: stack expects a non-empty TensorList

In [5]:
print("Getting num_passive for the UK")
df_uk_comb['passive_instances'] = df_uk_comb['combined_text'].apply(check_if_passive_en, dep_parser=dep_parser_en)
df_uk_comb.to_excel("../data/UK/uk_stanza_passive_900.xlsx")

Getting num_passive for the UK


# Spanish dependency parser

## Function


In [ ]:
dep_parser_es = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma,depparse')

def check_if_passive_es(sentence, dep_parser, method1=True):
    
    '''
    ADD DOCSTRING
    '''

    if sentence:

        doc = dep_parser(sentence)

        pr1 = ['det', 'nsubj', 'aux', 'root']
        pr2 = ['det', 'nsubj', 'advmod', 'aux', 'aux', 'root']
        pr3 = ['det', 'nsubj', 'amod', 'aux', 'root']
        pr4 = ['det', 'nsubj', 'aux', 'root', 'case', 'det']
        pr5 = "pass" #expletive passive


        sent_lst = []
        num_passive = 0
        num_passive_expl = 0
        
        for sent in doc.sentences:
            num_passive = 0
            num_passive_expl = 0

            sent_lst = []
            deprel_lst = []
            for word in sent.words:
                deprel = word.deprel
                deprel_lst.append(deprel)
                sent_lst.append(sent)

                if pr1 in deprel_lst or pr2 in deprel_lst or pr3 in deprel_lst or pr4 in deprel_lst:
                    num_passive += 1
                    return [num_passive, num_passive_expl, sent_lst]

                if pr5 in deprel:
                    num_passive_expl +=1
                    return [num_passive, num_passive_expl, sent_lst]
                

        return [num_passive, num_passive_expl, sent_lst]
        
    else:
        return [None, None, None]
            

2021-05-23 15:29:23 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |

2021-05-23 15:29:23 INFO: Use device: cpu
2021-05-23 15:29:23 INFO: Loading: tokenize
2021-05-23 15:29:23 INFO: Loading: mwt
2021-05-23 15:29:23 INFO: Loading: pos
2021-05-23 15:29:24 INFO: Loading: lemma
2021-05-23 15:29:24 INFO: Loading: depparse
2021-05-23 15:29:24 INFO: Done loading processors!


## Method 1

In [ ]:
df_mex_comb['combined_text'] = df_mex_comb['title'] + df_mex_comb['text']
print("Getting num_passive for Mexico")
df_mex_comb['passive_instances'] = df_mex_comb['combined_text'].apply(check_if_passive_es, dep_parser=dep_parser_es)
df_mex_comb.to_excel("../data/Mexico/mexico_stanza_passive_900.xlsx")

0    [0, 1, [[\n  {\n    "id": 1,\n    "text": "Lue...
1    [0, 1, [[\n  {\n    "id": 1,\n    "text": "Jau...
2    [0, 1, [[\n  {\n    "id": 1,\n    "text": "Las...
3    [0, 1, [[\n  {\n    "id": 1,\n    "text": "Aca...
4    [0, 1, [[\n  {\n    "id": 1,\n    "text": "Ay...
5    [0, 1, [[\n  {\n    "id": 1,\n    "text": "En"...
6    [0, 1, [[\n  {\n    "id": 1,\n    "text": "De"...
7    [0, 1, [[\n  {\n    "id": 1,\n    "text": "En"...
8    [0, 1, [[\n  {\n    "id": 1,\n    "text": "Los...
9    [0, 1, [[\n  {\n    "id": 1,\n    "text": "Hay...
Name: passive_instances, dtype: object